In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv
from datetime import datetime
import requests
import json
import pandas as pd
from typing import List, Dict, Any
import sqlite3
import praw
import asyncio
from openai import OpenAI
from agents import Agent, Runner, function_tool, trace, WebSearchTool
import gradio as gr

load_dotenv(override=True)

True

In [2]:
REDDIT_CLIENT_ID = os.getenv("REDDIT_CLIENT_ID")
REDDIT_CLIENT_SECRET = os.getenv("REDDIT_CLIENT_SECRET")
REDDIT_USER_AGENT = "RedditScraper/1.0"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT
)
Path("temp").mkdir(exist_ok=True)
Path("exports").mkdir(exist_ok=True)

# Structured data

In [3]:
# Structure JSON pour les paramètres utilisateur
USER_PARAMS_STRUCTURE = {
    "subreddit_name": "string",  # Nom du subreddit
    "num_posts": 10,             # Nombre de posts à scraper (max 50)
    "comments_limit": 10,        # Nombre de commentaires par post (max 50)
    "sort_criteria": "top",      # top, new, hot, best, rising
    "time_filter": "month",      # hour, day, week, month, year, all (pour top/rising)
    "confirmed": True,           # Validation utilisateur
    "timestamp": "2024-01-01 12:00:00",  # Date/heure de la demande
    "user_preferences": {        # Préférences utilisateur
        "language": "auto",      # Langue d'analyse
        "export_format": "all"   # Format d'export préféré
    }
}

In [4]:
# Structure JSON pour les données scrapées
SCRAPED_DATA_STRUCTURE = {
    "scraping_success": True,           # Booléen
    "subreddit": "string",              # Nom du subreddit
    "posts_count": 10,                  # Nombre de posts récupérés
    "total_comments": 100,              # Total des commentaires
    "scraped_at": "2024-01-01 12:00:00", # Date/heure du scraping
    "parameters_used": {                # Paramètres utilisés
        "sort_criteria": "top",         # Critère de tri
        "time_filter": "month",         # Filtre temporel
        "comments_limit": 10            # Limite de commentaires
    },
    "posts": [                          # Liste des posts
        {
            "title": "string",          # Titre du post
            "author": "string",         # Auteur
            "score": 100,               # Score (upvotes)
            "upvote_ratio": 0.95,       # Ratio upvote
            "num_comments": 50,         # Nombre de commentaires
            "created_utc": "2024-01-01 10:00:00", # Date création
            "url": "https://reddit.com/...", # URL du post
            "selftext": "string",       # Contenu du post
            "id": "string",             # ID du post
            "comments": [               # Liste des commentaires
                {
                    "author": "string", # Auteur du commentaire
                    "body": "string",   # Contenu du commentaire
                    "score": 10,        # Score du commentaire
                    "created_utc": "2024-01-01 10:30:00", # Date création
                    "id": "string"      # ID du commentaire
                }
            ]
        }
    ],
    "error_message": None               # Message d'erreur si échec
}

In [5]:
# Structure JSON pour l'analyse des douleurs
PAIN_ANALYSIS_STRUCTURE = {
    "analysis_success": True,           # Booléen
    "subreddit": "string",              # Nom du subreddit analysé
    "posts_analyzed": 10,               # Nombre de posts analysés
    "total_comments_analyzed": 100,     # Total des commentaires analysés
    "analysis_timestamp": "2024-01-01 12:00:00", # Date/heure de l'analyse
    "pain_categories": [                # Liste des catégories de douleurs
        {
            "pain_type": "string",      # Type de douleur (ex: "technique", "financier")
            "frequency": 5,             # Nombre de posts mentionnant cette douleur
            "avg_upvotes": 25.5,        # Moyenne des upvotes
            "avg_comments": 12.3,       # Moyenne des commentaires
            "avg_intensity": 7.2,       # Intensité émotionnelle moyenne (1-10)
            "score": 45.6,              # Score de priorité calculé
            "rank": 1,                  # Rang dans le classement
            "descriptions": [           # Exemples de descriptions
                "Description 1",
                "Description 2"
            ],
            "posts_affected": [         # IDs des posts concernés
                "post_id_1",
                "post_id_2"
            ]
        }
    ],
    "top_3_pains": [                   # Top 3 des douleurs prioritaires
        {
            "pain_type": "string",
            "score": 45.6,
            "frequency": 5,
            "avg_intensity": 7.2,
            "summary": "Résumé de la douleur"
        }
    ],
    "overall_summary": "string",        # Résumé global de l'analyse
    "solutions_found": [               # Commentaires proposant des solutions
        {
            "comment_id": "string",
            "post_id": "string",
            "author": "string",
            "solution_text": "string",
            "score": 10
        }
    ],
    "error_message": None              # Message d'erreur si échec
}

In [6]:
# Structure JSON pour les recommandations
RECOMMENDATIONS_STRUCTURE = {
    "recommendations_success": True,    # Booléen
    "subreddit": "string",              # Nom du subreddit
    "analysis_summary": "string",       # Résumé de l'analyse de l'Agent 3
    "recommendations_timestamp": "2024-01-01 12:00:00", # Date/heure
    "business_opportunities": [         # Opportunités business identifiées
        {
            "pain_type": "string",      # Type de douleur
            "pain_score": 45.6,         # Score de la douleur
            "opportunity_rank": 1,      # Rang de l'opportunité
            "solutions": [              # 3 solutions proposées
                {
                    "type": "saas",     # saas, digital_product, content, marketing
                    "title": "string",  # Titre de la solution
                    "description": "string", # Description détaillée
                    "complexity": "low", # low, medium, high
                    "estimated_cost": "string", # Coût estimé
                    "time_to_market": "string" # Temps de développement
                }
            ],
            "market_size": "string",    # Taille du marché estimée
            "competition_level": "low"  # low, medium, high
        }
    ],
    "top_3_opportunities": [           # Top 3 des meilleures opportunités
        {
            "pain_type": "string",
            "pain_score": 45.6,
            "best_solution": {
                "type": "saas",
                "title": "string",
                "description": "string"
            }
        }
    ],
    "export_available": True,          # Si les exports sont disponibles
    "stored_solutions_count": 5,       # Nombre de solutions stockées en SQLite
    "error_message": None              # Message d'erreur si échec
}

In [8]:
@function_tool
def init_solutions_database() -> Dict[str, str]:
    """
    Initialise la base de données SQLite pour stocker les commentaires avec solutions
    
    Returns:
        Dict avec le statut de l'initialisation
    """
    try:
        conn = sqlite3.connect('solutions.db')
        cursor = conn.cursor()
        
        # Créer la table des solutions
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS solutions (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                comment_id TEXT UNIQUE,
                post_id TEXT,
                author TEXT,
                solution_text TEXT,
                score INTEGER,
                pain_type TEXT,
                intensity INTEGER,
                subreddit TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        
        conn.commit()
        conn.close()
        
        return {
            "success": True,
            "message": "✅ Base de données initialisée avec succès"
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

@function_tool
def store_exceptional_solution(comment_data: str, pain_type: str, intensity: int) -> Dict[str, str]:
    """
    Stocke un commentaire exceptionnel proposant une solution
    
    Args:
        comment_data: Données du commentaire en JSON string
        pain_type: Type de douleur identifiée
        intensity: Intensité émotionnelle (1-10)
    
    Returns:
        Dict avec le statut du stockage
    """
    try:
        comment = json.loads(comment_data)
        
        conn = sqlite3.connect('solutions.db')
        cursor = conn.cursor()
        
        cursor.execute('''
            INSERT OR REPLACE INTO solutions 
            (comment_id, post_id, author, solution_text, score, pain_type, intensity, subreddit)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        ''', (
            comment.get('id'),
            comment.get('post_id'),
            comment.get('author'),
            comment.get('body'),
            comment.get('score', 0),
            pain_type,
            intensity,
            comment.get('subreddit', 'unknown')
        ))
        
        conn.commit()
        conn.close()
        
        return {
            "success": True,
            "message": f"✅ Solution stockée: {comment.get('author', 'Unknown')}"
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

@function_tool
def get_stored_solutions(subreddit: str = None, limit: int = 10) -> Dict[str, str]:
    """
    Récupère les solutions stockées en base
    
    Args:
        subreddit: Filtrer par subreddit (optionnel)
        limit: Nombre maximum de solutions à récupérer
    
    Returns:
        Dict avec les solutions récupérées
    """
    try:
        conn = sqlite3.connect('solutions.db')
        cursor = conn.cursor()
        
        if subreddit:
            cursor.execute('''
                SELECT * FROM solutions 
                WHERE subreddit = ? 
                ORDER BY score DESC, created_at DESC 
                LIMIT ?
            ''', (subreddit, limit))
        else:
            cursor.execute('''
                SELECT * FROM solutions 
                ORDER BY score DESC, created_at DESC 
                LIMIT ?
            ''', (limit,))
        
        rows = cursor.fetchall()
        conn.close()
        
        solutions = []
        for row in rows:
            solutions.append({
                "id": row[0],
                "comment_id": row[1],
                "post_id": row[2],
                "author": row[3],
                "solution_text": row[4],
                "score": row[5],
                "pain_type": row[6],
                "intensity": row[7],
                "subreddit": row[8],
                "created_at": row[9]
            })
        
        return {
            "success": True,
            "solutions": solutions,
            "count": len(solutions)
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

In [9]:
@function_tool
def export_final_report(analysis_data: str, recommendations_data: str, format_type: str = "pdf") -> Dict[str, str]:
    """
    Exporte le rapport final avec analyse et recommandations
    
    Args:
        analysis_data: Données de l'analyse (Agent 3) en JSON string
        recommendations_data: Données des recommandations (Agent 4) en JSON string
        format_type: Format d'export (pdf, csv, txt)
    
    Returns:
        Dict avec le statut et le chemin du fichier
    """
    try:
        # Parser les données JSON
        analysis = json.loads(analysis_data)
        recommendations = json.loads(recommendations_data)
        
        filename = f"rapport_final_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        if format_type == "pdf":
            return export_analysis_to_pdf(analysis, recommendations, filename)
        elif format_type == "csv":
            return export_analysis_to_csv(analysis, recommendations, filename)
        elif format_type == "txt":
            return export_analysis_to_txt(analysis, recommendations, filename)
        else:
            return {"success": False, "error": "Format non supporté"}
            
    except Exception as e:
        return {"success": False, "error": str(e)}

@function_tool
def export_exceptional_solutions(format_type: str = "pdf") -> Dict[str, str]:
    """
    Exporte uniquement les solutions exceptionnelles stockées en base
    
    Args:
        format_type: Format d'export (pdf, csv, txt)
    
    Returns:
        Dict avec le statut et le chemin du fichier
    """
    try:
        # Récupérer toutes les solutions
        solutions_result = get_stored_solutions(limit=1000)
        
        if not solutions_result["success"]:
            return {"success": False, "error": "Impossible de récupérer les solutions"}
        
        solutions = solutions_result["solutions"]
        filename = f"solutions_exceptionnelles_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        export_dir = Path("exports")
        export_dir.mkdir(exist_ok=True)
        
        if format_type == "txt":
            filepath = export_dir / f"{filename}.txt"
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(f"SOLUTIONS EXCEPTIONNELLES REDDIT - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("=" * 60 + "\n\n")
                f.write(f"Nombre de solutions: {len(solutions)}\n\n")
                
                for i, solution in enumerate(solutions, 1):
                    f.write(f"SOLUTION {i}:\n")
                    f.write(f"Auteur: {solution['author']}\n")
                    f.write(f"Subreddit: r/{solution['subreddit']}\n")
                    f.write(f"Score: {solution['score']}\n")
                    f.write(f"Type de douleur: {solution['pain_type']}\n")
                    f.write(f"Intensité: {solution['intensity']}/10\n")
                    f.write(f"Date: {solution['created_at']}\n")
                    f.write(f"Solution:\n{solution['solution_text']}\n")
                    f.write("-" * 40 + "\n\n")
            
            return {"success": True, "filepath": str(filepath), "message": f"✅ Solutions exportées en TXT: {filepath}"}
        
        elif format_type == "csv":
            import csv
            filepath = export_dir / f"{filename}.csv"
            with open(filepath, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(['Auteur', 'Subreddit', 'Score', 'Type de douleur', 'Intensité', 'Date', 'Solution'])
                for solution in solutions:
                    writer.writerow([
                        solution['author'],
                        solution['subreddit'],
                        solution['score'],
                        solution['pain_type'],
                        solution['intensity'],
                        solution['created_at'],
                        solution['solution_text']
                    ])
            
            return {"success": True, "filepath": str(filepath), "message": f"✅ Solutions exportées en CSV: {filepath}"}
        
        elif format_type == "pdf":
            from reportlab.lib.pagesizes import letter
            from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
            from reportlab.lib.styles import getSampleStyleSheet
            
            filepath = export_dir / f"{filename}.pdf"
            doc = SimpleDocTemplate(str(filepath), pagesize=letter)
            styles = getSampleStyleSheet()
            story = []
            
            title = Paragraph(f"SOLUTIONS EXCEPTIONNELLES REDDIT", styles['Title'])
            story.append(title)
            story.append(Spacer(1, 12))
            
            for i, solution in enumerate(solutions, 1):
                story.append(Paragraph(f"<b>Solution {i}:</b>", styles['Heading2']))
                story.append(Paragraph(f"Auteur: {solution['author']}", styles['Normal']))
                story.append(Paragraph(f"Subreddit: r/{solution['subreddit']}", styles['Normal']))
                story.append(Paragraph(f"Score: {solution['score']}", styles['Normal']))
                story.append(Paragraph(f"Solution: {solution['solution_text']}", styles['Normal']))
                story.append(Spacer(1, 12))
            
            doc.build(story)
            return {"success": True, "filepath": str(filepath), "message": f"✅ Solutions exportées en PDF: {filepath}"}
        
        else:
            return {"success": False, "error": "Format non supporté"}
            
    except Exception as e:
        return {"success": False, "error": str(e)}

@function_tool
def export_both_reports(analysis_data: str, recommendations_data: str, format_type: str = "pdf") -> Dict[str, str]:
    """
    Exporte le rapport final ET les solutions exceptionnelles
    
    Args:
        analysis_data: Données de l'analyse (Agent 3) en JSON string
        recommendations_data: Données des recommandations (Agent 4) en JSON string
        format_type: Format d'export (pdf, csv, txt)
    
    Returns:
        Dict avec le statut et les chemins des fichiers
    """
    try:
        # Exporter le rapport final
        report_result = export_final_report(analysis_data, recommendations_data, format_type)
        
        # Exporter les solutions exceptionnelles
        solutions_result = export_exceptional_solutions(format_type)
        
        if report_result["success"] and solutions_result["success"]:
            return {
                "success": True,
                "report_file": report_result["filepath"],
                "solutions_file": solutions_result["filepath"],
                "message": "✅ Rapport final et solutions exportés"
            }
        else:
            return {
                "success": False,
                "error": f"Erreur rapport: {report_result.get('error', 'OK')}, Erreur solutions: {solutions_result.get('error', 'OK')}"
            }
            
    except Exception as e:
        return {"success": False, "error": str(e)}

# Tools

#### Subreddit check

In [10]:
@function_tool
def check_subreddit_exists(subreddit_name: str) -> Dict[str, Any]:
    """
    Vérifie si un subreddit existe via l'API PRAW
    
    Args:
        subreddit_name: Nom du subreddit (sans le 'r/')
    
    Returns:
        Dict avec les informations du subreddit
    """
    try:
        # Utiliser PRAW pour accéder au subreddit
        subreddit = reddit.subreddit(subreddit_name)
        
        # Essayer d'accéder aux informations du subreddit
        # Cela va lever une exception si le subreddit n'existe pas
        subreddit_info = {
            "exists": True,
            "subreddit": subreddit_name,
            "subscribers": subreddit.subscribers,
            "description": subreddit.public_description,
            "title": subreddit.title,
            "is_private": subreddit.subreddit_type == 'private',
            "created_utc": datetime.fromtimestamp(subreddit.created_utc).strftime('%Y-%m-%d'),
            "url": f"https://reddit.com/r/{subreddit_name}"
        }
        
        return subreddit_info
        
    except Exception as e:
        return {
            "exists": False,
            "subreddit": subreddit_name,
            "error": str(e)
        }

In [11]:
@function_tool
def calculate_pain_score(frequency: int, avg_upvotes: float, avg_comments: float, avg_intensity: float) -> Dict[str, Any]:
    """
    Calcule le score de priorité d'une douleur utilisateur
    
    Args:
        frequency: Nombre de posts mentionnant cette douleur
        avg_upvotes: Moyenne des upvotes des posts concernés
        avg_comments: Moyenne des commentaires des posts concernés
        avg_intensity: Intensité émotionnelle moyenne (1-10)
    
    Returns:
        Dict avec le score calculé et les détails
    """
    try:
        # Formule de scoring
        score = (frequency * 0.4) + (avg_upvotes * 0.2) + (avg_comments * 0.1) + (avg_intensity * 0.3)
        
        # Normaliser l'intensité (1-10 vers 0-100)
        normalized_intensity = avg_intensity * 10
        
        # Calculer les composantes pour le détail
        frequency_component = frequency * 0.4
        upvotes_component = avg_upvotes * 0.2
        comments_component = avg_comments * 0.1
        intensity_component = normalized_intensity * 0.3
        
        return {
            "success": True,
            "total_score": round(score, 2),
            "components": {
                "frequency_component": round(frequency_component, 2),
                "upvotes_component": round(upvotes_component, 2),
                "comments_component": round(comments_component, 2),
                "intensity_component": round(intensity_component, 2)
            },
            "formula": "score = (frequency * 0.4) + (avg_upvotes * 0.2) + (avg_comments * 0.1) + (intensity * 0.3)",
            "input_values": {
                "frequency": frequency,
                "avg_upvotes": avg_upvotes,
                "avg_comments": avg_comments,
                "avg_intensity": avg_intensity
            }
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e)
        }

#### Reddit Scraper

In [13]:
@function_tool
def scrape_subreddit_posts(subreddit_name: str, num_posts: int = 10, sort_criteria: str = "top", comments_limit: int = 10, time_filter: str = "month") -> Dict[str, Any]:
    """
    Scrape les posts d'un subreddit selon les paramètres donnés
    
    Args:
        subreddit_name: Nom du subreddit
        num_posts: Nombre de posts à récupérer (max 50)
        sort_criteria: Critère de tri (top, new, hot, best, rising)
        comments_limit: Nombre de commentaires par post (max 50)
        time_filter: Période pour top/rising (hour, day, week, month, year, all)
    
    Returns:
        Dict avec les posts scrapés et métadonnées
    """
    try:
        # Limiter les valeurs
        num_posts = min(num_posts, 50)
        comments_limit = min(comments_limit, 50)
        
        # Vérifier que le subreddit existe
        subreddit = reddit.subreddit(subreddit_name)
        
        # Vérifier si le critère demandé existe
        original_criteria = sort_criteria
        fallback_used = False
        
        # Tester seulement le critère demandé
        if sort_criteria == "top" and hasattr(subreddit, 'top'):
            sort_method = subreddit.top
        elif sort_criteria == "new" and hasattr(subreddit, 'new'):
            sort_method = subreddit.new
        elif sort_criteria == "hot" and hasattr(subreddit, 'hot'):
            sort_method = subreddit.hot
        elif sort_criteria == "best" and hasattr(subreddit, 'best'):
            sort_method = subreddit.best
        elif sort_criteria == "rising" and hasattr(subreddit, 'rising'):
            sort_method = subreddit.rising
        else:
            # Fallback sur "new" si le critère demandé n'existe pas
            sort_criteria = "new"
            sort_method = subreddit.new
            fallback_used = True
        
        # Récupérer les posts
        posts_data = []
        
        try:
            if sort_criteria in ["top", "rising"]:
                posts = sort_method(limit=num_posts, time_filter=time_filter)
            else:
                posts = sort_method(limit=num_posts)
        except Exception as e:
            # Si le critère échoue, essayer "new"
            sort_criteria = "new"
            posts = subreddit.new(limit=num_posts)
            fallback_used = True
        
        for post in posts:
            # Récupérer les commentaires
            post.comments.replace_more(limit=5)
            
            comments_data = []
            for comment in post.comments.list()[:comments_limit]:
                if hasattr(comment, 'body') and comment.body:
                    comments_data.append({
                        "author": str(comment.author) if comment.author else "[deleted]",
                        "body": comment.body,
                        "score": comment.score,
                        "created_utc": datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                        "id": comment.id
                    })
            
            post_data = {
                "title": post.title,
                "author": str(post.author) if post.author else "[deleted]",
                "score": post.score,
                "upvote_ratio": post.upvote_ratio,
                "num_comments": post.num_comments,
                "created_utc": datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                "url": f"https://reddit.com{post.permalink}",
                "selftext": post.selftext[:1000] + "..." if len(post.selftext) > 1000 else post.selftext,
                "comments": comments_data,
                "id": post.id
            }
            
            posts_data.append(post_data)
        
        return {
            "success": True,
            "subreddit": subreddit_name,
            "sort_criteria": sort_criteria,
            "original_criteria": original_criteria,
            "time_filter": time_filter,
            "posts_count": len(posts_data),
            "comments_limit_per_post": comments_limit,
            "posts": posts_data,
            "scraped_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "fallback_used": fallback_used
        }
        
    except Exception as e:
        return {
            "success": False,
            "error": str(e),
            "subreddit": subreddit_name
        }

# Agents

#### Agent 1 : Interaction Utilisateur

In [17]:
agent_1_interaction = Agent(
    name="UserInteractionAgent",
    instructions="""Tu es un assistant spécialisé dans l’analyse de Reddit pour identifier 
    les points de douleur des entrepreneurs.

Ton rôle est de:
1. Saluer l'utilisateur avec politesse et professionnalisme.
2. Expliquer clairement ta mission : analyser un subreddit pour détecter les problèmes/frustrations récurrents.
3. Demander le nom dusubreddit à analyser.
4. Présenter les 5 critères de tri Reddit pour les posts, de façon concise et pédagogique :
   - "Top" → les posts avec le meilleur score sur une période (votes positifs - négatifs)
   - "New" → les posts les plus récents (ordre chronologique)
   - "Hot" → mélange du score + fraîcheur (post récent et populaire)
   - "Best" → pertinence + vote + réponse
   - "Rising" → posts récents qui gagnent rapidement en popularité
5. Demander les paramètres d'analyse :
   - Nombre de posts à analyser
   - Nombre de commentaires par post
   - Période souhaitée (pour top ou rising)
6. Si l'utilisateur ne sait pas, expliquer brièvement chaque paramètre.
7. Proposer des valeurs par défaut si nécessaire :

Paramètres par défaut:
- Nombre de posts: 10
- Nombre de commentaires par post: 10
- Critère: "top"
- Période: "month" (pour top/rising)

8.Valider les choix de l’utilisateur, reformuler les paramètres et demander confirmation.
9.Retourner les paramètres sous forme de JSON structuré, prêt à être utilisé par le scraper.
10. Une fois que l'utilisateur fournit le subreddit, demande-lui si c'est bien le bon 
et si les paramètres par défaut lui conviennent. S'il confirme, Handoff ensuite le json à l'agent ScrapingAgent.

STRUCTURE JSON À RETOURNER:
{json.dumps(USER_PARAMS_STRUCTURE, indent=2)}
""",
    tools=[
        WebSearchTool(),
        check_subreddit_exists
    ],
    handoffs=[agent_2_scraping],
    model="gpt-4o-mini"
)

#### Agent 2 : Scraping des Données

In [16]:
agent_2_scraping = Agent(
    name="ScrapingAgent",
    instructions=f"""Tu es un agent spécialisé dans le scraping de données Reddit.

Ton rôle est de:
1. Recevoir les paramètres de l'Agent 1 (subreddit, nombre de posts, nombre de commentaires, critère de tri, période)
2. Scraper les données du subreddit en respectant fidèlement ces paramètres en utilisant l'outil scrape_subreddit_posts
3. Vérifier que les données ont bien été récupérées (pas vides, structure correcte)
4. Une fois toutes les données scrappées, Retourner un objet JSON structuré et handoff à PainAnalysisAgent

IMPORTANT - RESPECTER LES PARAMÈTRES REÇUS:
- Utilise EXACTEMENT le critère de tri reçu (top, new, hot, best, rising)
- Utilise EXACTEMENT le nombre de posts reçu
- Utilise EXACTEMENT le nombre de commentaires reçu
- Utilise EXACTEMENT la période reçue (pour top/rising)
- Ne change JAMAIS ces paramètres, même si tu penses qu'un autre critère serait mieux

IMPORTANT - POUR LE HANDOFF:
Après avoir terminé le scraping et créé le JSON structuré, tu DOIS faire un handoff vers l'agent PainAnalysisAgent 
en lui passant EXACTEMENT tes données scrapées dans le format JSON suivant.


STRUCTURE JSON À RETOURNER:
{json.dumps(SCRAPED_DATA_STRUCTURE, indent=2)}

En cas d'erreur, retourner:
{{
    "scraping_success": false,
    "error_message": "Description de l'erreur",
    "subreddit": "nom_du_subreddit"
}}

Sois rigoureux, précis et respecte TOUJOURS les paramètres reçus.""",
    tools=[
        scrape_subreddit_posts
    ],
    handoffs=[agent_3_analysis],
    model="gpt-4o-mini"
)

#### Agent 3 : Analyse et Synthèse

In [15]:

agent_3_analysis = Agent(
    name="PainAnalysisAgent",
    instructions=f"""Tu es un expert analyste spécialisé dans l'identification des problèmes et 
    frustrations récurrents des utilisateurs.

Ton rôle est de:
1. Analyser le sentiment et l’intensité émotionnelle de chaque post et commentaire 
2. Identifier les douleurs récurrentes, quelle que soit la langue
3. Calculer le score de priorité de chaque douleur avec l'outil calculate_pain_score
4. Classer les douleurs par ordre de priorité
5. Repérer les commentaires exceptionnels proposant des solutions concrètes
6. Stocker ces solutions exceptionnelles dans la base SQLite avec l'outil store_exceptional_solution
7. Fournir une analyse structurée et complète
8. Retourner un objet JSON structuré et handoff à RecommendationsAgent

Critères pour une solution exceptionnelle:
- Score du commentaire > 10
- Propose une solution concrète et réalisable
- Solution détaillée et utile

IMPORTANT - POUR LE HANDOFF:
Après avoir terminé ton analyse et créé le JSON structuré, tu DOIS faire un handoff vers l'agent RecommendationsAgent 
en lui passant EXACTEMENT ton analyse dans le format JSON suivant.

STRUCTURE JSON À RETOURNER:
{json.dumps(PAIN_ANALYSIS_STRUCTURE, indent=2)}

En cas d'erreur, retourner:
{{
    "analysis_success": false,
    "error_message": "Description de l'erreur",
    "subreddit": "nom_du_subreddit"
}}

""",
    tools=[
        calculate_pain_score,
        init_solutions_database,
        store_exceptional_solution,
        get_stored_solutions
    ],
    handoffs=[agent_4_recommendations],
    model="gpt-4o-mini"
)

#### Agent 4 : Recommandations

In [14]:
agent_4_recommendations = Agent(
    name="RecommendationsAgent",
    instructions=f"""Tu es un expert en business development et création d'entreprises.
    Tu es créatif, ingénieux, réaliste et propose des solutions concrètes et réalisables.

Ton rôle est de:
1. Analyser les douleurs identifiées par l'Agent 3
2. Pour chaque douleur identifiée, propose 3 opportunités business parmis:
   - Solutions SaaS (priorité)
   - Produits digitaux
   - Création de contenu
   - Marketing/Formation
3. Pour chaque opportunité, précise :
   - Le type (saas, digital_product, content, marketing, etc.)
   - Un titre clair
   - Une description détaillée
   - Le niveau de complexité
   - Le coût estimé
   - Le temps de développement estimé
4. Classer les opportunités business selon leur potentiel (rentabilité + faisabilité)
5. Présente le tout de façon claire et conversationnelle dans le chat (pas en JSON brut)
6. Proposer les exports disponibles avec les différents outils :
   - export_final_report: Rapport complet (analyse + recommandations)
   - export_exceptional_solutions: Seulement les commentaires exceptionnels
   - export_both_reports: Les deux rapports
7. Rester disponible pour répondre aux questions de l'utilisateur ou générer un export


FORMAT DE RÉPONSE :
- Pour chaque douleur, liste les 3 opportunités business proposées, avec leurs détails
- À la fin, propose les options d'export disponibles
- Invite l'utilisateur à demander un export, poser des questions, ou relancer une analyse sur un autre subreddit

Exemple de structure :
"Pour la douleur : [Nom de la douleur]
1. [Opportunité 1] (SaaS) - Complexité: [niveau], Coût: [estimation], Temps: [estimation]
   Description : ...
2. [Opportunité 2] (Produit digital) - ...
   Description : ...
3. [Opportunité 3] (Marketing/Formation) - ...
   Description : ...

[... Répéter pour chaque douleur identifiée ...]

📊 EXPORTS DISPONIBLES :
- Rapport complet (PDF/CSV/TXT)
- Solutions exceptionnelles uniquement
- Les deux rapports

Que souhaitez-vous faire ? Voulez-vous un export, avez-vous des questions, ou souhaitez-vous analyser un autre subreddit ?"

RÈGLE :
- Reste conversationnel et disponible pour continuer l'échange.
- Si l'utilisateur veut analyser un autre subreddit, transfère la main à l'Agent 1 (UserInteractionAgent) et explique-le clairement à l'utilisateur.



""",
    tools=[
        get_stored_solutions,
        export_final_report,
        export_exceptional_solutions,
        export_both_reports
    ],
    model="gpt-4o-mini"
)

# TEST Gradio

In [21]:
# Interface Gradio avec async correct
import gradio as gr
from agents import Runner, trace

async def simple_chat(message, history):
    """
    Fonction de chat avec async
    """
    try:
        # Construire le contexte avec l'historique
        context = ""
        if history:
            for user_msg, assistant_msg in history:
                context += f"Humain: {user_msg}\n"
                context += f"Assistant: {assistant_msg}\n"
        
        # Ajouter le message actuel
        context += f"Humain: {message}\nAssistant: "
        
        # Lancer l'Agent 1
        result = await Runner.run(agent_1_interaction, context)
        
        # Extraire la réponse
        if hasattr(result, 'final_output'):
            response_text = str(result.final_output)
        else:
            response_text = str(result)
        
        return response_text
        
    except Exception as e:
        return f"❌ Erreur: {str(e)}"

# Interface simple avec gr.ChatInterface
demo = gr.ChatInterface(
    fn=simple_chat,
    title="🤖 Analyse Reddit - Identification des Douleurs Utilisateurs",
    description="Découvrez les problèmes récurrents sur Reddit et trouvez des opportunités business",
    examples=[
        "Je veux analyser le subreddit entrepreneur",
        "Peux-tu m'expliquer les critères de tri Reddit ?",
        "Analysez le subreddit programming avec 20 posts"
    ],
    theme=gr.themes.Soft()
)

# Lancer
demo.launch(share=False, quiet=True)

c:\Users\Junca\projects\Projet_Reddit\Backend\.venv\Lib\site-packages\gradio\chat_interface.py:339: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It appears that you are using PRAW in an asynchronous environment.
It is strongly recommended to use Async PRAW: https://asyncpraw.readt